`Dataframe` to `DocumentArray`...

In [21]:
from docarray import Document, DocumentArray
import sys

sys.path.append('C:/Users/lachy/Videos/LACHY/SRI Repo/bias-project-ML')
from data_preprocess import data_selection
import metrics

train_df, test_df = data_selection('C:/Users/lachy/Pictures/utkface/utkface/')

train_da = DocumentArray.from_dataframe(train_df)
test_da = DocumentArray.from_dataframe(test_df)

# for item in train_da:
#     item.summary()


C:\Users/lachy/Videos/LACHY/SRI Repo/bias-project-ML\data_preprocess.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['gender'][i]= GENDER_MAPPER[df['gender'][i]]
C:\Users/lachy/Videos/LACHY/SRI Repo/bias-project-ML\data_preprocess.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['race'][i]= RACE_MAPPER[df['race'][i]]


Initializing `CLIP`...

In [23]:
import torch
import clip
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

`CLIP` Fine-tuning...

In [24]:
import finetuner

finetuner.login()

`Race ` and ` Gender` Fine-tuning...

In [25]:
gender_pairs = DocumentArray() # initialize a DocumentArray as final training data.

prompt = 'This is a person of '
for doc in train_da:
    pair = Document()
    doc.uri = doc.tags['filepath']
    img_chunk = doc.load_uri_to_image_tensor(224, 224)
    img_chunk.modality = 'image'
    txt_chunk = Document(content=prompt + doc.tags['gender'] + ' gender and ' + doc.tags['race'] + ' race.' )
    txt_chunk.modality = 'text'
    pair.chunks.extend([img_chunk, txt_chunk])
    # add pair to pairs
    gender_pairs.append(pair)

# Lets see the first item of the pairs
gender_pairs[0]

📄 Document: fee1dbdb709c89e0e0974edc6fc908e0
└── 💠 Chunks
    ├── 📄 Document: 736f3346fb2d969a46615700f90cafcc
    │   ╭───────────┬──────────────────────────────────────────────────────────────────╮
    │   │ Attribute │ Value                                                            │
    │   ├───────────┼──────────────────────────────────────────────────────────────────┤
    │   │ tensor    │ <class 'numpy.ndarray'> in shape (224, 224, 3), dtype: uint8     │
    │   │ mime_type │ image/jpeg                                                       │
    │   │ uri       │ C:/Users/lachy/Pictures/utkface/utkface/100_0_0_201701122135009… │
    │   │ tags      │ {'filename': '100_0_0_20170112213500903.jpg.chip.jpg', 'gender': │
    │   │           │ 'male', 'age': 100, 'filepath':                                  │
    │   │           │ 'C:/Users/lachy/Pictures/utkface/utkface/100_0_0_20170112213500… │
    │   │           │ 'race': 'white'}                                                 │
    │   │ modality  │ image                                                            │
    │   ╰───────────┴──────────────────────────────────────────────────────────────────╯
    └── 📄 Document: 5a9048c818ce4b14130f83432b76ae01
        ╭───────────────┬──────────────────────────────────────────────────────────────╮
        │ Attribute     │ Value                                                        │
        ├───────────────┼──────────────────────────────────────────────────────────────┤
        │ text          │ This is a person of male gender and white race.              │
        │ modality      │ text                                                         │
        ╰───────────────┴──────────────────────────────────────────────────────────────╯

In [27]:
for doc in gender_pairs:
    doc.chunks[0].tags = None
gender_pairs[0]


TypeError: 'Document' object is not subscriptable

In [71]:
len(gender_pairs)

18967

Training `CLIP` model...

In [72]:
# run = finetuner.fit(
#     model='openai/clip-vit-base-patch32', # fine-tune CLIP
#     train_data=gender_pairs,   
#     learning_rate=1e-5,
#     loss='CLIPLoss'
# )

In [73]:
# for entry in run.stream_logs():
#     print(entry)

Saving the `artifact`

In [74]:
# artifact = run.save_artifact('/artifact/')

Saving `encoders`...

In [75]:
clip_text_encoder = finetuner.get_model(artifact='/artifact/elated-easley.zip', select_model='clip-text')
clip_image_encoder = finetuner.get_model(artifact='/artifact/elated-easley.zip', select_model='clip-vision')

Saving `images embeddings`...

In [76]:
for d in test_da:
    d.uri = d.tags['filepath']
images = finetuner.encode(model=clip_image_encoder, data=test_da)

Output()

Create `query texts` to evaluate the model...

In [77]:
#Race query texts
race_labels = ['black', 'white', 'asian', 'indian', 'other']
race_tks = []
for label in race_labels:
    race_tks.append('A photo of a person of ' + label + ' race.')

#Sex query texts
sex_labels = ['male', 'female']
sex_tks = []
for label in sex_labels:
    sex_tks.append('A photo of a person of ' + label + ' gender.')




Saving `sex texts` embeddings...

In [78]:
text_docs = DocumentArray()
for item in sex_tks:
    text_docs.append(Document(content=item))
text_docs[0].summary()
text_docs[1].summary()

finetuner.encode(model=clip_text_encoder, data=text_docs)

📄 Document: 9d67edaefe98997f2df0df1f52a033eb
╭──────────────────┬───────────────────────────────────────────────────────────╮
│ Attribute        │ Value                                                     │
├──────────────────┼───────────────────────────────────────────────────────────┤
│ text             │ A photo of a person of male gender.                       │
╰──────────────────┴───────────────────────────────────────────────────────────╯

📄 Document: 9cd7c486d955db8565e89d4450beebcf
╭─────────────────┬────────────────────────────────────────────────────────────╮
│ Attribute       │ Value                                                      │
├─────────────────┼────────────────────────────────────────────────────────────┤
│ text            │ A photo of a person of female gender.                      │
╰─────────────────┴────────────────────────────────────────────────────────────╯

Output()

╭────────────────── Documents Summary ───────────────────╮
│                                                        │
│   Type                   DocumentArrayInMemory         │
│   Length                 2                             │
│   Homogenous Documents   True                          │
│   Common Attributes      ('id', 'text', 'embedding')   │
│   Multimodal dataclass   False                         │
│                                                        │
╰────────────────────────────────────────────────────────╯
╭────────────────────── Attributes Summary ───────────────────────╮
│                                                                 │
│   Attribute   Data type      #Unique values   Has empty value   │
│  ─────────────────────────────────────────────────────────────  │
│   embedding   ('ndarray',)   2                False             │
│   id          ('str',)       2                False             │
│   text        ('str',)       2                False             │
│                                                                 │
╰─────────────────────────────────────────────────────────────────╯

Testing model for `sex predictions`...

In [79]:
images.match(text_docs, metric='cosine', limit=1)
y_true = [image.tags['gender'] for image in images]
y_predict = [image.matches[0].text.split(' ')[-2] for image in images]
labels = ['male', 'female']

## Detecting bias in finetuned `CLIP ` (sex results)

`Disparate impact`

In [80]:
metrics.disparate_impact(y_true, y_predict, labels)

DISPARATE IMPACT.
No disparate impact present


False

`Equalized odds`

In [81]:
metrics.equalized_odds(y_true, y_predict, labels)

EQUALIZED ODDS
Equalized odds


True

## `Sex` Evaluation Metrics

`Macro accuracy`

In [82]:
metrics.macro_accuracy(y_true, y_predict)

0.9911511179065123

`Macro F1`

In [83]:
metrics.macro_f1(y_true, y_predict, labels)

0.9909155932176048

`Confusion matrix`

In [84]:
metrics.confusion_matrix(y_true, y_predict, labels, plot=False)

array([[0.98627926, 0.01372074],
       [0.00397702, 0.99602298]])

## Detecting bias in finetuned `CLIP ` (race results)

Saving `race texts` embeddings...

In [85]:
text_docs = DocumentArray()
for item in race_tks:
    text_docs.append(Document(content=item))
text_docs[0].summary()
text_docs[1].summary()

finetuner.encode(model=clip_text_encoder, data=text_docs)

📄 Document: e568d62c8f6fab0964fa6916a60fd764
╭───────────────────┬──────────────────────────────────────────────────────────╮
│ Attribute         │ Value                                                    │
├───────────────────┼──────────────────────────────────────────────────────────┤
│ text              │ A photo of a person of black race.                       │
╰───────────────────┴──────────────────────────────────────────────────────────╯

📄 Document: 022716316ce12d2e1ce314110d114122
╭───────────────────┬──────────────────────────────────────────────────────────╮
│ Attribute         │ Value                                                    │
├───────────────────┼──────────────────────────────────────────────────────────┤
│ text              │ A photo of a person of white race.                       │
╰───────────────────┴──────────────────────────────────────────────────────────╯

Output()

╭────────────────── Documents Summary ───────────────────╮
│                                                        │
│   Type                   DocumentArrayInMemory         │
│   Length                 5                             │
│   Homogenous Documents   True                          │
│   Common Attributes      ('id', 'text', 'embedding')   │
│   Multimodal dataclass   False                         │
│                                                        │
╰────────────────────────────────────────────────────────╯
╭────────────────────── Attributes Summary ───────────────────────╮
│                                                                 │
│   Attribute   Data type      #Unique values   Has empty value   │
│  ─────────────────────────────────────────────────────────────  │
│   embedding   ('ndarray',)   5                False             │
│   id          ('str',)       5                False             │
│   text        ('str',)       5                False             │
│                                                                 │
╰─────────────────────────────────────────────────────────────────╯

Testing model for `race predictions`...

In [86]:
images.match(text_docs, metric='cosine', limit=1)
y_true = [image.tags['race'] for image in images]
y_predict = [image.matches[0].text.split(' ')[-2] for image in images]
labels = ['white', 'black', 'asian', 'indian', 'other']


`Disparate impact`




In [87]:
metrics.disparate_impact(y_true, y_predict, labels)

DISPARATE IMPACT.


TypeError: can only concatenate str (not "numpy.float64") to str

`Equalized odds`

In [ ]:
metrics.equalized_odds(y_true, y_predict, labels)

EQUALIZED ODDS
Equalized odds


True

## `Race` Evaluation Metrics

`Macro accuracy`

In [ ]:
metrics.macro_accuracy(y_true, y_predict)

0.9540244357402621

`Macro F1`

In [ ]:
metrics.macro_f1(y_true, y_predict, labels)

0.9372403386662264

`Confusion matrix`

In [ ]:
metrics.confusion_matrix(y_true, y_predict, labels, plot = False)

array([[0.94890873, 0.00793651, 0.0014881 , 0.0014881 , 0.04017857],
       [0.00552486, 0.99337017, 0.        , 0.        , 0.00110497],
       [0.        , 0.0014556 , 0.97962154, 0.00436681, 0.01455604],
       [0.00377358, 0.02515723, 0.00125786, 0.93081761, 0.03899371],
       [0.01769912, 0.04424779, 0.01179941, 0.00884956, 0.91740413]])